# 1. Python & Astropy tools


In [1]:
import numpy as np
import matplotlib.pyplot as plt
#bethermine_data = np.load('/data/projects/FIRESIDE/data/bethermine_cat_csv.npy', allow_pickle=True)
#line_table_data = np.load('/data/projects/FIRESIDE/data/line_table_cat_v9.npy', allow_pickle=True)

#!conda install astropy -y
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import astropy as ap
from astropy.visualization import hist
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord



from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# 2. Function LineFlux

In [2]:
def line_flux_at_z(lineLum, z):
    d=cosmo.luminosity_distance(z).to('m')
    linelum=lineLum.to("W")
    lineFlux=linelum/(4.*np.pi*d**2)
    return lineFlux

# 3. Function for on sky sensitivity

In [3]:
def sensScaled (sensitivity, tobsv, fov, area):
    '''
    fov is the field of view of telescope (area) in units of square degrees.

    sensitivity is the sensitivity of the telescope in units of Watts per square meter. 

    tobsv describes the total time spent observing.

    area is the total area of observation. 
    '''
    fov=fov.to(u.degree**2)
    sensitivity=sensitivity.to(u.W/u.m**2)
    tobsv=tobsv.to(u.hr)
    area=area.to(u.degree**2)

    sensScaled=sensitivity*np.sqrt(1*u.hr/(tobsv*(fov)/(area)))
    return sensScaled

                  
# 4. Function for redshifted wavelength

In [4]:
def redshifted_wavelengthFunc(z, waveRest, waveMin, waveMax):
     
    '''
    z is the redshift of the source observed. Z has no units!
   
    waveRest is the wavelength of emitted light of the source. The function converts all entered values to the units of 'm'.
    
    waveMin is the wavelength cutoff min value. The function converts all entered values to the units of 'm'.
    
    waveMax is the wavelength cutoff max value.The function converts all entered values to the units of 'm'.

    The entered units for this function don't matter, as long as they're covertable to a unit of length, e.g., 'nm' to 'm'.
    '''

    waveRest=waveRest.to(u.micron)
    waveMin=waveMin.to(u.micron)
    waveMax=waveMax.to(u.micron)

    redshifted_wavelength=waveRest*(1+z)
    #detect_galaxies = np.where((redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax),1,0)
    return redshifted_wavelength

#resultRedshift = redshifted_wavelengthFunc(line_table_data[0], 158*u.micron, 25*u.micron, 250*u.micron)
#print(resultRedshift)

# 5. Element input function

#old def ElementInput(Element):
    
    waveRestData= [
   157.74 * u.micron, 145.53 * u.micron, 121.90 * u.micron, 88.36 * u.micron, 63.18 * u.micron,
   57.32 * u.micron, 51.81 * u.micron, 18.70 * u.micron, 15.60 * u.micron, 12.80 * u.micron,
   25.98 * u.micron, 33.48 * u.micron, 34.82 * u.micron, 10.50 * u.micron, 7.70 * u.micron,
   11.30 * u.micron, 4.05 * u.micron, 12.37 * u.micron, 7.46 * u.micron, 
   14.30 * u.micron, 24.31 * u.micron
]
   
    Element_names = [
   '[CII] 158', '[OI] 145', '[NII] 122', '[OIII] 88', '[OI] 63',
   '[NIII] 57', '[OIII] 52', '[SIII] 19', '[NeIII] 16', '[NeII] 13',
   '[OIV] 26', '[SIII] 33', '[SII] 35', '[SIV] 11', 'PAH 7.7',
   'PAH 11.3', 'Bralpha', 'Hmalpha', 'Pfalpa', 'Halpha',
   '[NeV] 14', '[NeV] 24'
]
    IndeciesArray = np.where(np.array(Element_names) == Element)[0]
    if len(IndeciesArray) == 0:
        raise ValueError(f"Element '{Element}' not found.")
        
    elif len(IndeciesArray) != 0:
        
        
        #lineLum = line_table_data[:, np.where(np.array(Element_names) == Element)[0]+11]*u.solLum
        #print(np.where(np.array(Element_names) == Element)[0][0])
        line_colunm = np.where(np.array(Element_names) == Element)[0][0]
        waveRest = waveRestData[np.where(np.array(Element_names) == Element)[0][0]]
        print(waveRest)
    
    
        return lineLum, waveRest
        
location,waveRest=ElementInput('[CII] 158')
print(result[0], waveRest)  

In [5]:
def ElementInput(Element):
    
    waveRestData= [
   157.74 * u.micron, 145.53 * u.micron, 121.90 * u.micron, 88.36 * u.micron, 63.18 * u.micron,
   57.32 * u.micron, 51.81 * u.micron, 18.70 * u.micron, 15.60 * u.micron, 12.80 * u.micron,
   25.98 * u.micron, 33.48 * u.micron, 34.82 * u.micron, 10.50 * u.micron, 7.70 * u.micron,
   11.30 * u.micron, 4.05 * u.micron, 12.37 * u.micron, 7.46 * u.micron, 
   14.30 * u.micron, 24.31 * u.micron
]
   
    Assigned_Column_Element = {
                                '[CII] 158':11,
                                '[OI] 145':12, 
                                '[NII] 122':13,
                                '[OIII] 88':14,
                                '[OI] 63':15,
                                '[NIII] 57':16, 
                                '[OIII] 52':17, 
                                '[SIII] 19':18,
                                '[NeIII] 16':19,
                                '[NeII] 13':20,
                                '[OIV] 26':21,
                                '[SIII] 33':22,
                                '[SII] 35':23,
                                '[SIV] 11':24,
                                'PAH 7.7':25,
                                'PAH 11.3':26, 
                                'Bralpha':27,
                                'Hmalpha':28,
                                'Pfalpa':29,
                                'Halpha':30,
                                '[NeV] 14':31, 
                                '[NeV] 24':32 
                                }
   
    #if len(IndeciesArray) == 0:
    #    raise ValueError(f"Element '{Element}' not found.")
        
    #elif len(IndeciesArray) != 0:
        
        
        #lineLum = line_table_data[:, np.where(np.array(Element_names) == Element)[0]+11]*u.solLum
        #print(np.where(np.array(Element_names) == Element)[0][0])
    #    line_colunm = np.where(np.array(Element_names) == Element)[0][0]
    #    waveRest = waveRestData[np.where(np.array(Element_names) == Element)[0][0]]
    #    print(waveRest)
    line_index = Assigned_Column_Element['[CII] 158']
    waveRest = waveRestData[line_index-11]
    
    return line_index, waveRest
        
line_index,waveRest=ElementInput('[CII] 158')
print(line_index, waveRest)  

11 157.74 micron


In [6]:
Assigned_Column_Element = {
'[CII] 158':11,
'[OI] 145':12, 
'[NII] 122':13,
'[OIII] 88':14,
'[OI] 63':15,
'[NIII] 57':16, 
'[OIII] 52':17, 
'[SIII] 19':18,
'[NeIII] 16':19,
'[NeII] 13':20,
'[OIV] 26':21,
'[SIII] 33':22,
'[SII] 35':23,
'[SIV] 11':24,
'PAH 7.7':25,
'PAH 11.3':26, 
'Bralpha':27,
'Hmalpha':28,
'Pfalpa':29,
'Halpha':30,
'[NeV] 14':31, 
'[NeV] 24':32 
}

In [7]:
Assigned_Column_Element['[CII] 158']

11

# 6. Galaxy Function 
(combines all previous functions for galaxy detections)

In [8]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
#line_table_data = np.load('/data/projects/FIRESIDE/data/line_table_cat_v9.npy', allow_pickle=True)

    

In [9]:
    

def Detected_Galaxies(Element, data_table, waveMin=25*u.micron, waveMax=250*u.micron, sensitivity=10**-19*u.W/u.m**2, tobsv=1000*u.hr, fov=1.6*10**-3*u.degree**2, area=1.96*u.degree**2):

    line_index,waveRest=ElementInput(Element)
    NewSensScaled=sensScaled(sensitivity, tobsv, fov, area)
    NewLineFlux=line_flux_at_z(data_table[line_index]*u.solLum, data_table[0])
    redshifted_wavelength=redshifted_wavelengthFunc(data_table[0], waveRest, waveMin, waveMax)


    
     #area = 1.96 #in square degrees
    MaxArea=1.96*u.degree**2
    if area > MaxArea:
        ra_length = MaxArea/np.sqrt(MaxArea)
        factor=int(area/MaxArea)
        dec_angle=MaxArea/np.sqrt(MaxArea)
        
        raise ValueError('Sky area exceedes allowed fov. Area is limitied to 0 - 1.96 sq.degrees')
                         
    elif area <= MaxArea:

        factor=1
        ra_length=area/np.sqrt(area)
        dec_angle=area/np.sqrt(area)

    catIndices = np.where((dec_angle>=data_table[2]*u.degree) & 
                          (ra_length>=data_table[1]*u.degree) & 
                          (NewLineFlux>=NewSensScaled) & 
                          (redshifted_wavelength>=waveMin) & 
                          (redshifted_wavelength<=waveMax))[0]

    
        
    FluxOfGalaxiesDetected = NewLineFlux[catIndices]
    
    Total_Galaxies = factor*len(NewLineFlux[catIndices]) 


        
    return line_index, catIndices, Total_Galaxies, FluxOfGalaxiesDetected

    
    
        
    print('Indices:', catIndices, 'Galaxy detections:', Total_Galaxies, 'Flux of Galaxies Det.:', FluxOfGalaxiesDetected) 


#redshift is really row 1, but that corresponds to index 0  
#['REDSHIFT' , 'RA' ,'DEC','MHALO','MSTAR','QFLAG', 'SFR','MU','ISSB','UMEAN','LIR',
#  1            2     3     4        5       6        7    8     9     10       11     
# 'l[CII] 158','l[OI] 145','l[NII] 122','l[OIII] 88','l[OI] 63','l[NIII] 57','l[OIII] 52','l[SIII] 19',
#  12            13           14           15           16          17         18           19
# 'l[NeIII] 16','l[NeII] 13','l[OIV] 26', 'l[SIII] 33','l[SII] 35','l[SIV] 11','lPAH 7.7','lPAH 11.3','lBralpha','lHmalpha','lPfalpha',
#  20             21          22           23           24            25           26        27         28         29         30
#  'lHalpha','l[NeV] 14','l[NeV] 24',
#  31            32       33
#  '[CII] 158','[OI] 145','[NII] 122','[OIII] 88','[OI] 63','[NIII] 57','[OIII] 52','[SIII] 19',
#   34
#  '[NeIII] 16','[NeII] 13','[OIV] 26','[SIII] 33','[SII] 35','[SIV] 11','PAH 7.7','PAH 11.3','Bralpha','Hmalpha','Pfalpha',
#   'Halpha','[NeV] 14', '[NeV] 24'])

#OLD

def Detected_Galaxies(Element, waveMin=25*u.micron, waveMax=, sensitivity=, tobsv=, fov=, area=):

    location=ElementInput(Element)
    NewSensScaled=sensScaled(sensitivity, tobsv, fov, area)
    NewLineFlux=line_flux_at_z(ElementLineLumAndWaveRest[0]*u.solLum, z)
    redshifted_wavelength=redshifted_wavelengthFunc(z, ElementLineLumAndWaveRest[1], waveMin, waveMax)


    
     #area = 1.96 #in square degrees
    MaxArea=1.96*u.degree**2
    if area > MaxArea:
        ra_length = MaxArea/np.sqrt(MaxArea)
        factor=int(area/MaxArea)
        dec_angle=MaxArea/np.sqrt(MaxArea)
        
        raise ValueError('Sky area exceedes allowed fov. Area is limitied to 0 - 1.96 sq.degrees')
                         
    elif area <= MaxArea:

        factor=1
        ra_length=area/np.sqrt(area)
        dec_angle=area/np.sqrt(area)

    Total_Galaxies = factor*np.sum(np.where((dec_angle>=data_table[2]*u.degree) & (ra_length>=line_table_data[1]*u.degree) & (NewLineFlux>=NewSensScaled) & (redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax), 1, 0)) 
        
    FluxOfGalaxiesDetected = np.where((dec_angle>=data_table[2]*u.degree) & (ra_length>=line_table_data[1]*u.degree) & (NewLineFlux>=NewSensScaled) & (redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax), NewLineFlux, 0)

    catIndices = np.where((dec_angle>=data_table[2]*u.degree) & (ra_length>=line_table_data[1]*u.degree) & (NewLineFlux>=NewSensScaled) & (redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax))[0]
        
    return catIndices, Total_Galaxies, FluxOfGalaxiesDetected

    
    
        
    print('Indices:', catIndices, 'Galaxy detections:', Total_Galaxies, 'Flux of Galaxies Det.:', FluxOfGalaxiesDetected) 


#redshift is really row 1, but that corresponds to index 0  
#['REDSHIFT' , 'RA' ,'DEC','MHALO','MSTAR','QFLAG', 'SFR','MU','ISSB','UMEAN','LIR',
#  1            2     3     4        5       6        7    8     9     10       11     
# 'l[CII] 158','l[OI] 145','l[NII] 122','l[OIII] 88','l[OI] 63','l[NIII] 57','l[OIII] 52','l[SIII] 19',
#  12            13           14           15           16          17         18           19
# 'l[NeIII] 16','l[NeII] 13','l[OIV] 26', 'l[SIII] 33','l[SII] 35','l[SIV] 11','lPAH 7.7','lPAH 11.3','lBralpha','lHmalpha','lPfalpha',
#  20             21          22           23           24            25           26        27         28         29         30
#  'lHalpha','l[NeV] 14','l[NeV] 24',
#  31            32       33
#  '[CII] 158','[OI] 145','[NII] 122','[OIII] 88','[OI] 63','[NIII] 57','[OIII] 52','[SIII] 19',
#   34
#  '[NeIII] 16','[NeII] 13','[OIV] 26','[SIII] 33','[SII] 35','[SIV] 11','PAH 7.7','PAH 11.3','Bralpha','Hmalpha','Pfalpha',
#   'Halpha','[NeV] 14', '[NeV] 24'])

#3 6.1 Value input for combined detected galaxy function

In [10]:
result = Detected_Galaxies('[CII] 158', )
print(result)

TypeError: Detected_Galaxies() missing 1 required positional argument: 'data_table'

## 6.2 Histograms for final detected galaxy function

ax = resultRedshift[result[0]].value
plt.figure()
hist(ax, bins='knuth', histtype='stepfilled', alpha=0.7)
plt.xlabel("Observed Wavelength")
plt.ylabel("N of Galaxies")
plt.title("N of Galaxies Detected for Observed Wavelength ")
plt.show()


ax = line_table_data[0][result[0]]
plt.figure()
hist(ax, bins='knuth', histtype='stepfilled', alpha=0.7)
plt.xlabel("Redshift")
plt.ylabel("N of Galaxies")
plt.title("N of Galaxies Detected by Redshift")
plt.show()

1) add in the filtering for an area smaller than 1.96 square degrees. take the square of area and then add two filters to where function on the RA and DEC this for less then the max area.
2) then for survey area BIGGER then 1.96, you basically just calculate and return a factor that used to scale correctly the number of detected galaxies detected so if your area is 3.92 sq degrees then you should get 2x more detections
3) automating the function, instead giving explicity the rest wave and line luminosities, have us selet the name of the line and automatically return the correct wavelengths and line fluxes from the catalog as functions

 # aaaaaaa
 

In [ ]:
%%writefile galaxy_functions.py

import numpy as np
import matplotlib.pyplot as plt




%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import astropy as ap
from astropy.visualization import hist
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord



from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

def line_flux_at_z(lineLum, z):
    d=cosmo.luminosity_distance(z).to('m')
    linelum=lineLum.to("W")
    lineFlux=linelum/(4.*np.pi*d**2)
    return lineFlux

def sensScaled (sensitivity, tobsv, fov, area):
    '''
    fov is the field of view of telescope (area) in units of square degrees.

    sensitivity is the sensitivity of the telescope in units of Watts per square meter. 

    tobsv describes the total time spent observing.

    area is the total area of observation. 
    '''
    fov=fov.to(u.degree**2)
    sensitivity=sensitivity.to(u.W/u.m**2)
    tobsv=tobsv.to(u.hr)
    area=area.to(u.degree**2)

    sensScaled=sensitivity*np.sqrt(1*u.hr/(tobsv*(fov)/(area)))
    return sensScaled

def redshifted_wavelengthFunc(z, waveRest, waveMin, waveMax):
     
    '''
    z is the redshift of the source observed. Z has no units!
   
    waveRest is the wavelength of emitted light of the source. The function converts all entered values to the units of 'm'.
    
    waveMin is the wavelength cutoff min value. The function converts all entered values to the units of 'm'.
    
    waveMax is the wavelength cutoff max value.The function converts all entered values to the units of 'm'.

    The entered units for this function don't matter, as long as they're covertable to a unit of length, e.g., 'nm' to 'm'.
    '''

    waveRest=waveRest.to(u.micron)
    waveMin=waveMin.to(u.micron)
    waveMax=waveMax.to(u.micron)

    redshifted_wavelength=waveRest*(1+z)
    #detect_galaxies = np.where((redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax),1,0)
    return redshifted_wavelength

resultRedshift = redshifted_wavelengthFunc(line_table_data[0], 158*u.micron, 25*u.micron, 250*u.micron)
print(resultRedshift)

def ElementInput(Element):
    
    waveRestData= [
   157.74 * u.micron, 145.53 * u.micron, 121.90 * u.micron, 88.36 * u.micron, 63.18 * u.micron,
   57.32 * u.micron, 51.81 * u.micron, 18.70 * u.micron, 15.60 * u.micron, 12.80 * u.micron,
   25.98 * u.micron, 33.48 * u.micron, 34.82 * u.micron, 10.50 * u.micron, 7.70 * u.micron,
   11.30 * u.micron, 4.05 * u.micron, 12.37 * u.micron, 7.46 * u.micron, 
   14.30 * u.micron, 24.31 * u.micron
]
   
    Element_names = [
   '[CII] 158', '[OI] 145', '[NII] 122', '[OIII] 88', '[OI] 63',
   '[NIII] 57', '[OIII] 52', '[SIII] 19', '[NeIII] 16', '[NeII] 13',
   '[OIV] 26', '[SIII] 33', '[SII] 35', '[SIV] 11', 'PAH 7.7',
   'PAH 11.3', 'Bralpha', 'Hmalpha', 'Pfalpa', 'Halpha',
   '[NeV] 14', '[NeV] 24'
]
    IndeciesArray = np.where(np.array(Element_names) == Element)[0]
    if len(IndeciesArray) == 0:
        raise ValueError(f"Element '{Element}' not found.")
        
    elif len(IndeciesArray) != 0:
        
        
        lineLum = line_table_data[:, np.where(np.array(Element_names) == Element)[0]+11]*u.solLum
        waveRest = waveRestData[np.where(np.array(Element_names) == Element)[0][0]]
    
    
        return lineLum, waveRest


def Detected_Galaxies(z, waveMin, waveMax, sensitivity, tobsv, fov, area, Element):

    ElementLineLumAndWaveRest=ElementInput(Element)
    NewSensScaled=sensScaled(sensitivity, tobsv, fov, area)
    NewLineFlux=line_flux_at_z(ElementLineLumAndWaveRest[0], z)
    redshifted_wavelength=redshifted_wavelengthFunc(z, ElementLineLumAndWaveRest[1], waveMin, waveMax)


    
     #area = 1.96 #in square degrees
    MaxArea=1.96*u.degree**2
    if area > MaxArea:
        ra_length = MaxArea/np.sqrt(MaxArea)
        factor=int(area/MaxArea)
        dec_angle=MaxArea/np.sqrt(MaxArea)
        
        raise ValueError('Sky area exceedes allowed fov. Area is limitied to 0 - 1.96 sq.degrees')
                         
    elif area <= MaxArea:

        factor=1
        ra_length=area/np.sqrt(area)
        dec_angle=area/np.sqrt(area)

    Total_Galaxies = factor*np.sum(np.where((dec_angle>=line_table_data[2]*u.degree) & (ra_length>=line_table_data[1]*u.degree) & (NewLineFlux>=NewSensScaled) & (redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax), 1, 0)) 
        
    FluxOfGalaxiesDetected = np.where((dec_angle>=line_table_data[2]*u.degree) & (ra_length>=line_table_data[1]*u.degree) & (NewLineFlux>=NewSensScaled) & (redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax), NewLineFlux, 0)

    catIndices = np.where((dec_angle>=line_table_data[2]*u.degree) & (ra_length>=line_table_data[1]*u.degree) & (NewLineFlux>=NewSensScaled) & (redshifted_wavelength>=waveMin) & (redshifted_wavelength<=waveMax))[0]
        
    return catIndices, Total_Galaxies, FluxOfGalaxiesDetected

    
    
        
    print('Indices:', catIndices, 'Galaxy detections:', Total_Galaxies, 'Flux of Galaxies Det.:', FluxOfGalaxiesDetected) 


result = Detected_Galaxies(line_table_data[0], 25*u.micron, 250*u.micron, 10**-19*u.W/u.m**2, 1000*u.hr, 1.6*10**-3*u.degree**2, 1.96*u.degree**2, '[CII] 158')